# Fine-tuning Llama 3.2 1B Model

This notebook demonstrates how to fine-tune the Llama 3.2 1B model using Hugging Face, PEFT, and LoRA for efficient fine-tuning.

In [ ]:

!pip install transformers==4.44.2
!pip install datasets
!pip install accelerate
!pip install peft
!pip install bitsandbytes
!pip install wandb


## 1. Load the Pre-trained Model and Tokenizer

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device_map="auto")


## 2. Prepare Dataset for Fine-tuning

In [ ]:

from datasets import load_dataset

# Load a sample dataset for fine-tuning
dataset = load_dataset("your_dataset_name")  # Replace with your dataset


## 3. Set Up the Trainer and Training Arguments

In [ ]:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)


## 4. Fine-tune the Model

In [ ]:

# Begin training
trainer.train()


## 5. Save the Fine-tuned Model and Push to Hugging Face Hub

In [ ]:

# Save the model locally
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

# Optionally, push to Hugging Face Hub
model.push_to_hub("your_hf_model_name")


## 6. Run Inference with Fine-tuned Model

In [ ]:

# Sample inference
inputs = tokenizer("Your test input", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
